LLM verwenden um bundestags reden zu einem bestimmten Thema zuzuordnen: 
oLLAMA funktioniert nicht. - zuu langfsam

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from ollama import chat, ChatResponse

# Pfade zu den CSV-Dateien
input_csv_path = "v5_merged_speeches.csv"
output_csv_path = "v6_labeled.csv"

# Liste der Labels
labels = []

# Prompt-Vorlage für das Modell
prompt_template = """
Klassifiziere die folgende Bundestagsrede entsprechend der folgenden Themen: {labels}.
Deine Antwort darf NUR aus einer Themen bestehen und soll GENAU GLEICH geschrieben werden.
Deine Antwort soll so formatiert sein: "Thema."
Rede: "{rede}"
"""

# Daten laden
df = pd.read_csv(input_csv_path)

# Sicherstellen, dass die Spalte 'label' existiert
if "label" not in df.columns:
    df["label"] = None

# Startindex für die Verarbeitung ermitteln
start_index = df[df["label"].isna()].index.min()

if start_index is None:
    print("Alle Zeilen sind bereits gelabelt. Verarbeitung beendet.")
else:
    print(f"Beginne mit der Verarbeitung ab Zeile {start_index}.")

# Verarbeitung der Reden mit Fortschrittsanzeige
for index, row in tqdm(df.iloc[start_index:].iterrows(), total=df.shape[0] - start_index, desc="Verarbeitung der Reden"):
    rede = row["cleaned_rede"]

    # Aktuelle Statusanzeige
    print(f"\n=== Verarbeite Zeile {index} ===")
    print(f"Text der Rede: {rede[:100]}...")  # Nur die ersten 100 Zeichen anzeigen

    # Prompt f
    # 
    # ür die aktuelle Rede erstellen
    prompt = prompt_template.format(labels=", ".join(labels), rede=rede)

    # Anfrage an das Ollama-Modell
    try:
        print("Sende Anfrage an Ollama...")
        response: ChatResponse = chat(model='phi3.5', messages=[{'role': 'user', 'content': prompt}])
        label = response.message.content.strip()
        print(f"Antwort von Ollama: {label}")
        df.at[index, "label"] = label

    except Exception as e:
        print(f"Fehler bei der Verarbeitung der Rede in Zeile {index}: {e}")
        df.at[index, "label"] = "Fehler"

    # Zwischenspeicherung nach 50 Iterationen
    if (index - start_index + 1) % 50 == 0:
        df.to_csv(output_csv_path, index=False)
        print(f"Zwischenspeicherung nach {index - start_index + 1} Zeilen.")

# Endgültige Speicherung
df.to_csv(output_csv_path, index=False)
print(f"Die Datei mit den Labels wurde gespeichert: {output_csv_path}")


In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import google.generativeai as genai

# .env-Datei laden
load_dotenv()

# Google API-Schlüssel laden
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("API-Schlüssel nicht gefunden. Bitte in der .env-Datei angeben.")
genai.configure(api_key=api_key)

# Initialisiere das Modell
model = genai.GenerativeModel("gemini-1.5-flash-8b")

# Pfade zu den CSV-Dateien
input_csv_path = "v6_labeled.csv"
output_csv_path = "v7_labeled_additional_info.csv"

# Liste der Labels
labels = [
    "Arbeitsmarktpolitik (Mindestlohn, Tarifverträge, Arbeitslosengeld)",
    "Rentenpolitik (Altersversorgung, Rentenniveau)",
    "Sozialversicherung (Kranken-, Pflegeversicherung, Unfallversicherung)",
    "Familienpolitik (Kinderbetreuung, Elternzeitregelungen, Kinderschutz, Jugendhilfe)",
    "Gleichstellung (Inklusion, Barrierefreiheit, Frauenrechte, Anti-Diskriminierung)",
    "Außenpolitik (Diplomatische, internationale Beziehungen, Friedensmissionen)",
    "Entwicklungshilfe (Internationale Armutsbekämpfung, Humanitäre Hilfe)",
    "EU-Politik (Europäische Integration, Zusammenarbeit mit EU-Institutionen)",
    "Bildungspolitik (Schulbildung, Berufsausbildung, Hochschulen)",
    "Forschung (Wissenschaftsförderung)",
    "Digitalisierung (Infrastruktur, Datenpolitik, Innovationstechnologien)",
    "Verbraucherschutz (Lebensmittelstandards, Datenschutz)",
    "Tier und Naturschutz (Biodiversität, Waldschutz, Tierrechte)",
    "Klimapolitik (Nachhaltigkeit, CO₂-Reduktion, Emissionshandel)",
    "Agrarpolitik",
    "Nukleare Sicherheit (Endlagerung von Atommüll, Sicherheit von Kernkraftwerken)",
    "Gesundheits- und Pharmapolitik (Krankenversorgung, Prävention, Arzneimittelsicherheit)",
    "Pandemiebekämpfung (Infektionsschutz, Impfstrategien)",
    "Industrie- und Handelspolitik (Freihandelsabkommen, Strukturwandel)",
    "Steuerpolitik (Einkommens- und Unternehmensbesteuerung)",
    "Haushaltspolitik (Staatsschulden und Ausgabenpolitik, Schuldenbremse)",
    "Medienpolitik (Rundfunkgesetzgebung, Pressefreiheit)",
    "Energiepolitik (Erneuerbare Energien, Effizienzsteigerungen, Versorgungssicherheit)",
    "Verkehrspolitik (Infrastruktur für Straßen, Schienen, Flugsicherheit, Schifffahrtsverkehr)",
    "Mobilitätswende (Elektroautos, öffentlicher Verkehr, öffentliche Verkehrsmittel)",
    "Kulturförderung (Denkmalschutz, Kulturelle Bildung, Sportförderung)",
    "Verteidigung (Bundeswehr, Rüstungspolitik, Cybersecurity)",
    "Migration und Integration (Asylrecht, Flüchtlingspolitik)",
    "Sicherheitsfragen (Polizei, Terrorismusbekämpfung)",
    "Gesetzgebung",
    "Verfassungsfragen (Grundrechte, Föderalismus, Menschenrechte)",
    "Stadtentwicklung (Wohnungsbau, Bauwesen, Infrastrukturprojekte)",
    "Tourismusförderung (Reisen und Gastgewerbe)"
]

# Prompt-Vorlage für die Label-Überprüfung und Stichwortgenerierung
review_prompt_template = """
Die folgende Bundestagsrede wurde als {label} klassifiziert. Prüfe, ob diese Klassifikation korrekt ist, und gebe zwei alternative Klassifikationen an. Hier siehst du noch einmal mögliche andere Labels:
{labels}
Zusätzlich generiere 5 Schlüsselwörter, die den Inhalt der Rede näher beschreiben.

Rede: "{rede}"

Das Ergebniss muss zwingend wie folgt formatiert sein:
[Label1, alternatives Label2, alternatives Label3, Schlüsselwört1, Schlüsselwört2, Schlüsselwört3, Schlüsselwört4, Schlüsselwört5]
"""

# Daten laden
df = pd.read_csv(input_csv_path)

# Sicherstellen, dass die Spalten 'label' und 'output' existieren
if "label" not in df.columns:
    raise ValueError("Die Spalte 'label' fehlt. Bitte die erste Iteration durchführen.")
if "output" not in df.columns:
    df["output"] = None

# Startindex für die Verarbeitung ermitteln
start_index = df[df["output"].isna()].index.min()

if start_index is None:
    print("Alle Zeilen wurden bereits überprüft. Verarbeitung beendet.")
else:
    print(f"Beginne mit der Überprüfung ab Zeile {start_index}.")

# Verarbeitung der Reden mit Fortschrittsanzeige
for index, row in tqdm(df.iloc[start_index:].iterrows(), total=df.shape[0] - start_index, desc="Überprüfung der Labels"):
    rede = row["text"]
    label = row["label"]

    # Prompt für die aktuelle Rede erstellen
    review_prompt = review_prompt_template.format(label=label, rede=rede, labels=labels)

    # Anfrage an Google Generative AI
    try:
        response = model.generate_content(review_prompt)
        output = response.text.strip()
        df.at[index, "output"] = output
        print(f"Antwort von Google Generative AI (Zeile {index}): {output}")

    except Exception as e:
        print(f"Fehler bei der Überprüfung der Rede in Zeile {index}: {e}")
        df.at[index, "output"] = "[Fehler, Fehler, Fehler, Fehler]"

    # Zwischenspeicherung nach 50 Iterationen
    if (index - start_index + 1) % 50 == 0:
        df.to_csv(output_csv_path, index=False)
        print(f"Zwischenspeicherung nach {index - start_index + 1} Zeilen.")

# Endgültige Speicherung
df.to_csv(output_csv_path, index=False)
print(f"Die Datei mit den überprüften Labels wurde gespeichert: {output_csv_path}")

